<a href="https://colab.research.google.com/github/Lokesh1854/Short-Term-Load-Forecasting/blob/main/parallel_cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, LSTM, concatenate, Dense, Dropout
from keras.models import Model

In [ ]:
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, LSTM, concatenate, Dense, Dropout, Activation
from keras.models import Model

# LSTM input
lstm_input = Input(shape=(48,1))  # Assuming input shape of (time_steps, features)

# LSTM layers
lstm_output_1 = LSTM(100, activation='relu', return_sequences=True)(lstm_input)
lstm_output_2 = LSTM(100, activation='relu',)(lstm_output_1)
lstm_output = Dense(1,activation='relu')(lstm_output_2)


# CNN input
cnn_input = Input(shape=(48,9))  # Assuming input shape of (additional_features, time_steps)

# CNN layers

conv_1 = Conv1D(filters=128, kernel_size=12, padding='same')(cnn_input)
act_1 = keras.layers.Activation('relu') (conv_1)
conv_2 = Conv1D(filters=64, kernel_size=6, strides=2, padding='same') (act_1)
act_2 = keras.layers.Activation('relu') (conv_2)

pool = keras.layers.GlobalAveragePooling1D() (act_2)

dc = Dense(256,activation='relu') (pool)
drop = Dropout(0.2)(dc)
dc1 = Dense(128,activation='relu') (drop)
dc2 = Dense(64,activation='relu') (dc1)
dc3 = Dense(32,activation='relu') (dc2)
dc4 = Dense(16,activation='relu') (dc3)
dc5 = Dense(8,activation='relu') (dc4)
dc6 = Dense(1,activation='relu')(dc5)

# Concatenate LSTM and CNN outputs
concatenated = concatenate([lstm_output, dc3])

# Dense layers for combined output

output_time_series = Dense(1, activation='relu')(concatenated)  # Output for LSTM

# Define model with multiple inputs and multiple outputs
model_p = Model(inputs=[lstm_input, cnn_input], outputs=[output_time_series])

# Compile the model
model_p.compile(optimizer='adam', loss=['mape'])

# Print model summary
model_p.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_14 (InputLayer)       [(None, 48, 9)]              0         []                            
                                                                                                  
 conv1d_8 (Conv1D)           (None, 48, 128)              13952     ['input_14[0][0]']            
                                                                                                  
 activation_4 (Activation)   (None, 48, 128)              0         ['conv1d_8[0][0]']            
                                                                                                  
 conv1d_9 (Conv1D)           (None, 24, 64)               49216     ['activation_4[0][0]']        
                                                                                            

In [ ]:
# creating time_steps of 1 day -- 24 x 6
look_back = 2*24

def prepare_cnn_data(data,look_back):
  X,Y = [],[]
  for i in range(len(data)-look_back):
    X.append(np.array(data.iloc[i:i+look_back,:-1]))
    Y.append(np.array(data.iloc[i+look_back,-1]))
  return np.array(X),np.array(Y)

In [ ]:
look_back = 2*24

def prepare_lstm_data(data,look_back):
  X_lstm,Y_lstm = [],[]
  for i in range(len(data)-look_back):
    X_lstm.append(np.array(data.iloc[i:i+look_back,-1]))
    Y_lstm.append(np.array(data.iloc[i+look_back,-1]))
  return np.array(X_lstm),np.array(Y_lstm)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
cluster_path = "/content/drive/MyDrive/STLF/DataSet/Cluster 0"

mape_err = []

for csvFile in os.listdir(cluster_path):
  csvPath = os.path.join(cluster_path,csvFile)
  df = pd.read_csv(csvPath)

  time_attributes = pd.DataFrame(df['timestamp'])
  time_attributes["timestamp"] = pd.to_datetime(time_attributes["timestamp"])

  time_attributes["hour"] = time_attributes["timestamp"].dt.hour                          # represents hour of the day
  time_attributes["minute"] = time_attributes["timestamp"].dt.minute                      # represents minute
  time_attributes["minute"] = time_attributes["minute"] + (60*time_attributes["hour"])

  time_attributes.drop("timestamp",axis=1,inplace=True)
  time_attributes.drop("hour",axis=1,inplace=True)

  df = pd.concat([time_attributes,df],axis=1)
  df.drop("timestamp",axis=1,inplace=True)

  X,Y = prepare_cnn_data(df,look_back)
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

  X_lstm,Y_lstm = prepare_lstm_data(df,look_back)
  X_lstm_train, X_lstm_test, Y_lstm_train, Y_lstm_test = train_test_split(X_lstm, Y_lstm, test_size=0.1, random_state=42)

  model_p.fit([X_lstm_train, X_train], Y_train , epochs=25, batch_size=32, validation_split=0.1)

  mape_err.append([csvFile,model_p.evaluate([X_lstm_test,X_test],Y_test)])

Epoch 1/25
531/531 [==============================] - 76s 135ms/step - loss: 19.1949 - val_loss: 13.5773
Epoch 2/25
531/531 [==============================] - 58s 109ms/step - loss: 14.5731 - val_loss: 12.8479
Epoch 3/25
531/531 [==============================] - 53s 99ms/step - loss: 14.3384 - val_loss: 15.8469
Epoch 4/25
531/531 [==============================] - 52s 97ms/step - loss: 13.8520 - val_loss: 13.0424
Epoch 5/25
531/531 [==============================] - 52s 97ms/step - loss: 13.6716 - val_loss: 15.5683
Epoch 6/25
531/531 [==============================] - 50s 95ms/step - loss: 13.6798 - val_loss: 12.2360
Epoch 7/25
531/531 [==============================] - 49s 93ms/step - loss: 13.3038 - val_loss: 16.1237
Epoch 8/25
531/531 [==============================] - 50s 94ms/step - loss: 13.1098 - val_loss: 16.4362
Epoch 9/25
531/531 [==============================] - 49s 93ms/step - loss: 13.0955 - val_loss: 13.7230
Epoch 10/25
531/531 [==============================] - 50s 94m

In [ ]:
os.chdir(cluster_path)

In [ ]:
mape_err

[['block_16.csv', 13.817168235778809],
 ['block_27.csv', 15.282452583312988],
 ['block_4.csv', 13.416032791137695],
 ['block_9.csv', 12.530925750732422]]

In [ ]:
model_p.save('cnn_lstm_parrallel')

In [ ]:
new_df = pd.read_csv('/content/drive/MyDrive/STLF/DataSet/modified_dataset/block_13.csv')

time_attributes = pd.DataFrame(new_df['timestamp'])
time_attributes["timestamp"] = pd.to_datetime(time_attributes["timestamp"])

time_attributes["hour"] = time_attributes["timestamp"].dt.hour                          # represents hour of the day
time_attributes["minute"] = time_attributes["timestamp"].dt.minute                      # represents minute
time_attributes["minute"] = time_attributes["minute"] + (60*time_attributes["hour"])

time_attributes.drop("timestamp",axis=1,inplace=True)
time_attributes.drop("hour",axis=1,inplace=True)

new_df = pd.concat([time_attributes,new_df],axis=1)
new_df.drop("timestamp",axis=1,inplace=True)

X_new,Y_new = prepare_cnn_data(new_df,look_back)
X_new_lstm,Y_new_lstm = prepare_lstm_data(new_df,look_back)
model_p.evaluate([X_new_lstm,X_new],Y_new)

581/581 [==============================] - 18s 30ms/step - loss: 14.0335


14.033499717712402